In [175]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Consolidate Multiple Files

### 1) Elezioni Camera e Senato 2018

In [176]:
data_dir = r'raw'
paths = ['camera_2018', 'senato_2018']

final_dfs = []

for p in paths:
    
    print(p)
    
    fullpath = os.path.join(data_dir, p)
    files = os.listdir(fullpath)
    
    temp_files = []
    
    for f in files:
        
        temp = pd.read_csv(os.path.join(fullpath,f), sep = ";")
        
        # Forward fill for multi-paty candidates
        temp = temp.fillna(method = 'ffill')
        temp['Elezione'] = p        
        temp = temp[['Ente', 'Candidato', 'Liste/Gruppi', 'Voti lista', 'Elezione']]
        
        temp_files.append(temp)
        
    # Concatenate temp files
    temp_elezioni = pd.concat(temp_files)
    
    final_dfs.append(temp_elezioni)

parlamento2018 = pd.concat(final_dfs)

camera_2018
senato_2018


### 2) Regionali 2015

In [177]:
data_dir = r'raw'
paths = ['regionali_2015']

final_dfs = []

for p in paths:
    
    print(p)
    
    fullpath = os.path.join(data_dir, p)
    files = os.listdir(fullpath)
    
    temp_files = []
    
    for f in files:
        
        temp = pd.read_csv(os.path.join(fullpath,f), sep = ";")
        cols = ['Ente', 'Candidato', 'Liste/Gruppi', 'Voti lista', 'Voti candidato']
        temp['Candidato'] = temp['Candidato'].fillna(method = 'ffill')
        temp = temp[cols]
        temp_files.append(temp)
        
    # Concatenate temp files
    temp_elezioni = pd.concat(temp_files)

regionali2015 = temp_elezioni.copy()

regionali_2015


### 3) Europee 2019

In [178]:
data_dir = r'raw'
paths = ['europee_2019']

final_dfs = []

for p in paths:
    
    print(p)
    
    fullpath = os.path.join(data_dir, p)
    files = os.listdir(fullpath)
    
    temp_files = []
    
    for f in files:
        
        temp = pd.read_csv(os.path.join(fullpath,f), sep = ";").reset_index()
        temp.columns = ['Ente', 'Liste/Gruppi', 'Voti', '1', '2']
        temp.drop(['1', '2'], 1, inplace = True)
        temp_files.append(temp)
        
europee2019 = pd.concat(temp_files)

europee_2019


# Analytics

### Functions

In [134]:
def get_comuni_campania_geo():

    geo_av = pd.read_html('http://www.dossier.net/utilities/coordinate-geografiche/provincia-avellino.htm')[0].iloc[1:]
    geo_av.columns = ['Ente', 'Latitudine', 'Longitudine']
    geo_av['Ente'] = [x.upper() for x in geo_av['Ente']]

    geo_bn = pd.read_html('http://www.dossier.net/utilities/coordinate-geografiche/provincia-benevento.htm')[0].iloc[1:]
    geo_bn.columns = ['Ente', 'Latitudine', 'Longitudine']
    geo_bn['Ente'] = [x.upper() for x in geo_bn['Ente']]

    geo_sa = pd.read_html('http://www.dossier.net/utilities/coordinate-geografiche/provincia-salerno.htm')[0].iloc[1:]
    geo_sa.columns = ['Ente', 'Latitudine', 'Longitudine']
    geo_sa['Ente'] = [x.upper() for x in geo_sa['Ente']]

    geo_na = pd.read_html('http://www.dossier.net/utilities/coordinate-geografiche/provincia-napoli.htm')[0].iloc[1:]
    geo_na.columns = ['Ente', 'Latitudine', 'Longitudine']
    geo_na['Ente'] = [x.upper() for x in geo_na['Ente']]

    geo_ce = pd.read_html('http://www.dossier.net/utilities/coordinate-geografiche/provincia-caserta.htm')[0].iloc[1:]
    geo_ce.columns = ['Ente', 'Latitudine', 'Longitudine']
    geo_ce['Ente'] = [x.upper() for x in geo_ce['Ente']]

    comuni_campania = pd.concat([geo_av, geo_bn, geo_sa, geo_na, geo_ce])
    
    return comuni_campania


def generate_pivot(df, columns, values, geo):
    
    pivot = pd.pivot_table(df, 
               index = 'Ente',
               columns = columns,
               values = values, 
               aggfunc = np.sum
              ).fillna(0)

    pivot['VOTI TOTALI'] = pivot.sum(axis = 1)

    new_cols_order = []

    for c in pivot.columns[:-1]:

        new_cols_order.append(c)
        new_col = '{}_PERC'.format(c)
        new_cols_order.append(new_col)

        pivot[new_col] = pivot[c] / pivot['VOTI TOTALI']

    pivot = pivot[new_cols_order].reset_index()
    
    pivot = pd.merge(pivot, geo, on = 'Ente', how = 'left')
    
    return pivot

### Analysis

In [135]:
geo_comuni_italiani =  get_comuni_campania_geo()

In [136]:
pivot_parlamento = generate_pivot(parlamento2018, 'Liste/Gruppi', 'Voti lista', geo_comuni_italiani)

In [137]:
pivot_regionali = generate_pivot(regionali2015, 'Liste/Gruppi', 'Voti lista', geo_comuni_italiani)

In [138]:
pivot_europee = generate_pivot(europee2019, 'Liste/Gruppi', 'Voti', geo_comuni_italiani)

In [139]:
with pd.ExcelWriter('output/dati_elettorali.xlsx') as writer:
    parlamento2018.to_excel(writer, sheet_name='parlamento2018', index = False)
    regionali2015.to_excel(writer, sheet_name='regionale2015', index = False)
    europee2019.to_excel(writer, sheet_name='europee2019', index = False)
    pivot_parlamento.to_excel(writer, sheet_name='pivot_parlamento', index = False)
    pivot_regionali.to_excel(writer, sheet_name='pivot_regionali', index = False)
    pivot_europee.to_excel(writer, sheet_name='pivot_europee', index = False)

### Analysis Gruppi

In [140]:
gruppi_parl = {'ITALIA EUROPA INSIEME': 'Bacino',
               'CIVICA POPOLARE LORENZIN': 'Bacino',
               'LIBERI E UGUALI': 'Bacino',
               'POTERE AL POPOLO!': 'Bacino',
               'PER UNA SINISTRA RIVOLUZIONARIA': 'Bacino',
               '+EUROPA': 'Bacino',
               'PARTITO COMUNISTA': 'Bacino',
               '10 VOLTE MEGLIO': 'Bacino',
               'LISTA DEL POPOLO PER LA COSTITUZIONE': 'Bacino',
               'MOVIMENTO 5 STELLE': '5 Stelle'
              }

parlamento2018['Gruppo'] = parlamento2018['Liste/Gruppi'].map(gruppi_parl)
parlamento2018['Gruppo'].fillna('Altro', inplace = True)

pivot_parlamento = generate_pivot(
    parlamento2018, 'Gruppo', 'Voti lista', geo_comuni_italiani)

In [141]:
gruppi_reg = {'DAVVERO-VERDI': 'Bacino',
              'CAMPANIA IN RETE': 'Bacino',
              'PARTITO SOCIALISTA ITALIANO': 'Bacino',
              'SINISTRA AL LAVORO': 'Bacino',
              'MO! LISTA CIVICA CAMPANIA': 'Bacino',
              'MOVIMENTO 5 STELLE BEPPEGRILLO.IT': '5 Stelle'
             }

regionali2015['Gruppo'] = regionali2015['Liste/Gruppi'].map(gruppi_reg)
regionali2015['Gruppo'].fillna('Altro', inplace = True)

pivot_regionali = generate_pivot(
    regionali2015, 'Gruppo', 'Voti lista', geo_comuni_italiani)

In [142]:
gruppi_eur = {"+EUROPA - ITALIA IN COMUNE - PDE ITALIA": 'Bacino',
                "LA SINISTRA": 'Bacino',
                "EUROPA VERDE": 'Bacino',
                "PARTITO ANIMALISTA": 'Bacino',
                "PARTITO COMUNISTA": 'Bacino',
                "PARTITO PIRATA": 'Bacino',
                "MOVIMENTO 5 STELLE": '5 Stelle'}

europee2019['Gruppo'] = europee2019['Liste/Gruppi'].map(gruppi_eur)
europee2019['Gruppo'].fillna('Altro', inplace = True)

pivot_europee = generate_pivot(
    europee2019, 'Gruppo', 'Voti', geo_comuni_italiani)

In [143]:
# Valori medii elezioni per ente

elect_all = pd.concat([pivot_parlamento, pivot_europee])
elect_all['Latitudine'] = elect_all['Latitudine'].astype(float)
elect_all['Longitudine'] = elect_all['Longitudine'].astype(float)

elect_all_agg = elect_all.groupby('Ente').mean().reset_index().dropna()

elect_all_agg.sort_values(by = 'Bacino_PERC', ascending = False).head(10)

,Ente,5 Stelle,5 Stelle_PERC,Altro,Altro_PERC,Bacino,Bacino_PERC,Latitudine,Longitudine
330,PETRURO IRPINO,59.5,0.273252,77.5,0.479420,63.0,0.247329,41.032224,14.798313
299,NOVI VELIA,470.0,0.253058,727.5,0.503766,421.0,0.243176,40.224973,15.287512
244,MAGLIANO VETERE,144.5,0.260378,215.5,0.509055,110.0,0.230567,40.347901,15.237148
399,SALZA IRPINA,273.5,0.383507,276.0,0.429712,136.5,0.186782,40.921426,14.887654
408,SAN GIOVANNI A PIRO,518.5,0.178153,1627.0,0.641040,486.0,0.180807,40.051426,15.451889
545,VILLAMAINA,263.5,0.351847,334.0,0.467720,153.0,0.180433,40.970481,15.090944
162,CONCA DEI MARINI,153.5,0.279332,256.5,0.541557,69.5,0.179112,40.617878,14.570232
66,CAIRANO,50.0,0.212920,116.5,0.608101,56.0,0.178979,40.894848,15.367601
367,PUGLIANELLO,380.5,0.280548,623.5,0.542531,272.5,0.176921,41.224397,14.450612
428,SAN NICOLA BARONIA,225.5,0.304378,322.0,0.518900,139.5,0.176722,41.059537,15.200000


In [144]:
# Crea score moltiplicando voti medi bacino per percentuale voti
elect_all_agg['Score'] = elect_all_agg['Bacino'] * elect_all_agg['Bacino_PERC']
elect_all_agg.sort_values(by = 'Score', ascending = False).head(10)

,Ente,5 Stelle,5 Stelle_PERC,Altro,Altro_PERC,Bacino,Bacino_PERC,Latitudine,Longitudine,Score
295,NAPOLI,156554.0,0.454064,151065.0,0.451056,31918.5,0.094880,40.839997,14.252871,3028.418423
397,SALERNO,32270.5,0.310324,51273.0,0.569629,10877.5,0.120047,40.677957,14.765912,1305.811985
38,AVELLINO,15412.0,0.331907,22568.5,0.539105,5801.0,0.128987,40.915168,14.795488,748.254659
112,CASERTA,22288.0,0.390154,24662.5,0.518339,4722.5,0.091506,41.075415,14.332195,432.138182
354,PORTICI,18292.5,0.465301,14461.0,0.425793,3939.0,0.108905,40.814051,14.339019,428.978434
358,POZZUOLI,24840.0,0.465042,20125.5,0.445160,4383.0,0.089797,40.823745,14.121622,393.581720
178,EBOLI,8017.0,0.315987,12240.0,0.566296,3005.0,0.117718,40.615333,15.058759,353.741948
139,CAVA DE' TIRRENI,15421.0,0.372605,19413.0,0.536038,3277.0,0.091357,40.700340,14.685950,299.375550
405,SAN GIORGIO A CREMANO,16623.5,0.492004,12290.5,0.417645,2731.5,0.090351,40.829170,14.334193,246.792744
51,BENEVENTO,18919.5,0.400666,20808.5,0.526322,3258.5,0.073012,41.130448,14.781174,237.909312


# Heatmaps

In [90]:
import os
import folium
from folium.plugins import HeatMap

print(folium.__version__)

0.9.1


In [145]:
cols = ['Ente', 'Bacino', 'Bacino_PERC', 'Score', 'Latitudine', 'Longitudine']

sub = elect_all_agg[cols].dropna()
sub.head(2)

,Ente,Bacino,Bacino_PERC,Score,Latitudine,Longitudine
4,ACERNO,109.0,0.057778,6.297774,40.738759,15.057815
5,ACERRA,1519.5,0.050667,76.989035,40.951722,14.377204


In [146]:
sub_map = sub.copy()
sub_map.columns = ['Ente', 'Voti_Medi', 'Percentuale_Media', 'Score', 'Latitudine', 'Longitudine']
sub_map.head(2)

,Ente,Voti_Medi,Percentuale_Media,Score,Latitudine,Longitudine
4,ACERNO,109.0,0.057778,6.297774,40.738759,15.057815
5,ACERRA,1519.5,0.050667,76.989035,40.951722,14.377204


In [147]:
m = folium.Map([sub['Latitudine'].mean(), 
                sub['Longitudine'].mean()],  
                zoom_start=8)

sub_map['Score_Bucket'] = pd.qcut(sub_map['Score'], 3, labels = ['Low', 'Medium', 'High'])

for p in sub_map.itertuples():
    
    lat = p.Latitudine
    lon = p.Longitudine
    ente = p.Ente
    score = p.Score
    perc = p.Percentuale_Media
    voti = p.Voti_Medi
    score_bucket = p.Score_Bucket
    
    pop = pd.DataFrame({'Comune': [ente], 'Voti Medi': [voti], 
                        'Percentuale Media': [perc], 'Score': [score] })
    html = pop.to_html(classes='table table-striped table-hover table-condensed table-responsive')
    
    if score_bucket == 'High':
    
        folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        fill=True,
        popup=folium.Popup(html),
        color='green'
            ).add_to(m)
        
    elif score_bucket == 'Medium':
        
        folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        fill=True,
        popup=folium.Popup(html),
        color='orange'
            ).add_to(m)
    
    else:
        
        folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        fill=True,
        popup=folium.Popup(html),
        color='red'
            ).add_to(m)
        
    

In [150]:
m.save('mappa_voti.html')

In [148]:
m

In [149]:
sub_map

,Ente,Voti_Medi,Percentuale_Media,Score,Latitudine,Longitudine,Score_Bucket
4,ACERNO,109.0,0.057778,6.297774,40.738759,15.057815,Low
5,ACERRA,1519.5,0.050667,76.989035,40.951722,14.377204,High
6,AFRAGOLA,1178.5,0.036895,43.480436,40.921037,14.307217,High
7,AGEROLA,253.0,0.049351,12.485754,40.638759,14.539870,Medium
8,AGROPOLI,903.5,0.067687,61.154963,40.350583,14.989517,High
...,...,...,...,...,...,...,...
549,VITULANO,158.0,0.069631,11.001680,41.176259,14.642370,Medium
550,VITULAZIO,343.0,0.071243,24.436188,41.163921,14.216643,High
551,VOLLA,682.5,0.050192,34.255931,40.879556,14.343926,High
552,VOLTURARA IRPINA,367.5,0.156992,57.694657,40.879543,14.917249,High


In [156]:
# Calcolo Allocazione Budget per High Propensity

sub_a = sub_map[sub_map['Score_Bucket'] == 'High'].copy()
sub_b = sub_map[sub_map['Score_Bucket'] != 'High'].copy()

sub_a['Score_Ratio'] = sub_a['Score'] / sub_a['Score'].sum()
sub_b['Score_Ratio'] = 0

fin_df = pd.concat([sub_a, sub_b])

fin_df.to_excel('Report_Voti.xlsx', index = False)

## Studio 5 Stelle

In [191]:
sub_parl = pivot_parlamento[['Ente', '5 Stelle', '5 Stelle_PERC', 'Latitudine', 'Longitudine']].copy()
sub_parl.dropna(inplace = True)

sub_europee = pivot_europee[['Ente', '5 Stelle', '5 Stelle_PERC']].copy()
sub_europee.columns = ['Ente', '5 Stelle_eu', '5 Stelle_PERC_eu']

fivedf = pd.merge(sub_parl, sub_europee, on = 'Ente', how = 'left')

In [192]:
# Calcola Delta

fivedf['Variazione_5_Stelle'] = (fivedf['5 Stelle_PERC_eu'] - fivedf['5 Stelle_PERC']) / fivedf['5 Stelle_PERC']

In [197]:
# Aggiungi Delta 5 Stelle al file calcolato

fin_df = pd.merge(fin_df, fivedf[['Ente', 'Variazione_5_Stelle']], on = 'Ente', how = 'left')

fin_df = fin_df[['Ente', 'Voti_Medi', 'Percentuale_Media', 'Variazione_5_Stelle', 'Score', 'Score_Bucket', 
       'Score_Ratio', 'Latitudine', 'Longitudine']]
fin_df.to_excel('Report_Voti.xlsx', index = False)

In [194]:
fivedf.sort_values(by = '5 Stelle', ascending = False)

,Ente,5 Stelle,5 Stelle_PERC,Latitudine,Longitudine,5 Stelle_eu,5 Stelle_PERC_eu,Variazione_5_Stelle
289,NAPOLI,194887,0.509568,40.8399968,14.2528707,118221,0.398560,-0.217847
207,GIUGLIANO IN CAMPANIA,60476,0.568314,40.9286009,14.1971619,14503,0.434287,-0.235833
391,SALERNO,51601,0.385871,40.6779567,14.7659122,12940,0.234778,-0.391565
109,CASORIA,45596,0.630127,40.9054455,14.2900745,17510,0.498775,-0.208453
511,TORRE DEL GRECO,41775,0.568035,40.7872187,14.3680286,9281,0.422247,-0.256652
...,...,...,...,...,...,...,...,...
516,TORTORELLA,89,0.185031,40.1425687,15.6065239,21,0.118644,-0.358789
383,ROMAGNANO AL MONTE,80,0.215054,40.6283677,15.4571498,26,0.152941,-0.288824
62,CAIRANO,78,0.237805,40.8948477,15.3676007,22,0.188034,-0.209292
526,VALLE DELL'ANGELO,76,0.257627,40.3443275,15.3698902,44,0.271605,0.054256


In [161]:
pivot_europee.head()

,Ente,5 Stelle,5 Stelle_PERC,Altro,Altro_PERC,Bacino,Bacino_PERC,Latitudine,Longitudine
0,ACERNO,334,0.332669,614,0.611554,56,0.055777,40.7387592,15.0578148
1,ACERRA,8305,0.498889,7351,0.441581,991,0.059530,40.9517222,14.3772037
2,AFRAGOLA,6019,0.385759,8955,0.573928,629,0.040313,40.9210375,14.3072168
3,AGEROLA,454,0.157748,2269,0.788395,155,0.053857,40.6387592,14.5398703
4,AGROPOLI,1825,0.232959,5459,0.696834,550,0.070207,40.3505832,14.9895165
